In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pandas.io import json
csv1=pd.read_csv(r"C:\ML Project\ML-final-project\Files\ids_0.csv")
csv2=pd.read_csv(r"C:\ML Project\ML-final-project\Files\ids_1.csv")
csv3=pd.read_csv(r"C:\ML Project\ML-final-project\Files\ids_2.csv")
json1=pd.read_json(r"C:\ML Project\ML-final-project\Files\ids_3.json",lines=True)
json2=pd.read_json(r"C:\ML Project\ML-final-project\Files\ids_4.json",lines=True)
json3=pd.read_json(r"C:\ML Project\ML-final-project\Files\ids_7.json",lines=True)
json4=pd.read_json(r"C:\ML Project\ML-final-project\Files\ids_9.json",lines=True)
json5=pd.read_json(r"C:\ML Project\ML-final-project\Files\ids_10.json",lines=True)
parq1=pd.read_parquet(r"C:\ML Project\ML-final-project\Files\ids_5.parquet")
parq2=pd.read_parquet(r"C:\ML Project\ML-final-project\Files\ids_6.parquet")
parq3=pd.read_parquet(r"C:\ML Project\ML-final-project\Files\ids_8.parquet")
parq4=pd.read_parquet(r"C:\ML Project\ML-final-project\Files\ids_11.parquet")

list_of_files=[csv1,csv2,csv3,json1,json2,json3,json4,json5,parq1,parq2,parq3,parq4]


In [ ]:
# Check for unique label
for files in list_of_files:
  print(files[' Label'].unique())

In [ ]:
# Fix column names for all variables before concatenation:
for df in list_of_files:
    df.columns = df.columns.str.strip()

# Merge all the files into one
df_master = pd.concat(list_of_files, ignore_index=True)
df_master.head()

In [ ]:
df_master['Label'].unique()

In [ ]:
df_master.info()

In [ ]:
# Remove rows containing the label "HeartbLeed"
df_master = df_master[df_master['Label'] != 'Heartbleed']
print(df_master['Label'].unique())
df_master.shape[0]

In [ ]:
# Replace Dos attacks with "ATTACK"
for labels in df_master['Label'].unique():
  if labels != 'BENIGN':
    df_master['Label'] = df_master['Label'].replace(labels, 'ATTACK')

print("labels after removing heartbleed and renaming Dos attacks are:",df_master['Label'].unique())

In [ ]:
df_master.info()

In [ ]:
# Remove INF values
numeric_df = df_master.select_dtypes(include=np.number)

# 2. Count rows containing infinite values
rows_with_inf = np.isinf(numeric_df).any(axis=1).sum()

print(f"Total number of flows (rows) containing at least one Inf value: {rows_with_inf}")

In [ ]:
# Create a copy of merged DataFrame
df_clean = df_master.copy()

# Replace all infinite values (Inf) with NaN
df_clean.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop all rows that contain ANY NaN value (which now includes the Inf values)
df_clean.dropna(inplace=True)

# Verification
flows_removed = len(df_master) - len(df_clean)
print(f"Total rows removed due to infinite and null values: {flows_removed}")

In [ ]:
# Convert 'BENIGN' to 0 and 'ATTACK' to 1 for scaling and training
df_clean['Label'] = df_clean['Label'].replace(['BENIGN', 'ATTACK'], [0, 1])

print("Labels converted:" ,df_clean['Label'].unique())

print("Rows and columns:",df_clean.shape)

In [ ]:
df_clean.info()

In [ ]:
# Prepare dataset for scaling
X = df_clean.drop('Label', axis=1).copy()
y = df_clean['Label'].copy()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaling successful!")

